In [16]:
from os import path, listdir
import sys
sys.path.append("..")

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from src.tracking import *

In [17]:
test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow_from_directory('../DB/test', batch_size = 3568, class_mode = 'categorical', target_size = (227, 227))

# On récupère les images d'un batch de la taille du dataset
i = 0
for image, lab in test_generator:
    if i == 0: break # sinon le generator continue sa boucle (??)

Found 3568 images belonging to 5 classes.


# Tracking vidéo
En partant d'une première détection de l'objet d'intérêt qui prend la forme d'une boite englobante que l'on suppose correcte, on va suivre l'objet sur les frames suivantes.

Pour chaque frame, on prend la boîte englobante de la frame précédente, on va créer un set de nouvelles boîtes potentielles. On va ensuite récupérer le "patch" de l'image associé à chaque nouvelle boîte englobante et effectuer une prediction à l'aide du modèle précédemment créé. On conservera comme nouvelle boîte celle qui fournira la prédiction la plus précise (pourcentage le plus élevé).

In [18]:
# on charge le modèle
model = tf.keras.models.load_model('../models/best_models', custom_objects=None, compile=True, options=None)

In [19]:
# on vérifie les performances sur le jeu de test
model.evaluate(image, lab)

112/112 [==============================] - 36s 318ms/step - loss: 0.0565 - accuracy: 0.9846 - categorical_accuracy: 0.9846 - recall: 0.9835 - precision: 0.9851


[0.05651376023888588,
 0.9845852255821228,
 0.9845852255821228,
 0.9834641218185425,
 0.9851207137107849]

In [ ]:
ious = []
i = 0
# Pour toutes les vidéos qui sont dans le jeu de test
for file in listdir("../GT_test"):
    if file.endswith(".txt"):
        i += 1
        print(f'Video {i} on {len(listdir("../GT_test"))}')
        
        m = re.search('(.+)_._bboxes\.txt', file)
        name = m.group(1)
        
        path_video = path.join('..', 'VIDEOS', name + '.mp4')
        path_boxes = path.join('..', 'GT_test', file)
        
        label = get_label(path_video)
        predictions, iou = tracking(model, path_video, path_boxes, label, update=None, save_fig=True)
        mean_iou = np.mean(list(iou.values()))
        ious.append(mean_iou)
        print(f'Mean IoU: {np.mean(ious_move_to_data)}')
            
print(f'Total Mean IoU: {np.mean(ious)}')

Video 1 on 27
Loading 186 frames


 68%|██████▊   | 126/186 [04:35<04:24,  4.41s/it]

## Move-to-data
Nous implémentons ici la méthode d'apprentissage incrémental "move-to-data" décrite dans _Move-to-Data: A new Continual Learning approach with Deep CNNs, Application for image-class recognition_

Pour chaque nouvelle image, on met à jour les poids de la dernière couche du réseau en appliquant la formule suivante 
$$w_j′=w_j+(||w_j||∗\frac{v_i}{||v_i||} − w_j)*\epsilon$$

In [ ]:
ious_move_to_data = []
i = 0
# Pour toutes les vidéos qui sont dans le jeu de test
for file in listdir("../GT_test"):
    if file.endswith(".txt"):
        i += 1
        print(f'Video {i} on {len(listdir("../GT_test"))}')
        
        m = re.search('(.+)_._bboxes\.txt', file)
        name = m.group(1)
        
        path_video = path.join('..', 'VIDEOS', name + '.mp4')
        path_boxes = path.join('..', 'GT_test', file)
        
        label = get_label(path_video)
        
        # on lance le tracking en activant le move-to-data, on met à jour le réseau toute les 10 images
        predictions, iou = tracking(model, path_video, path_boxes, label, update='move-to-data', epsilon=0.0001, n_update=10, save_fig=True)
        mean_iou = np.mean(list(iou.values()))
        ious_move_to_data.append(mean_iou)
        print(f'Mean IoU: {np.mean(ious_move_to_data)}')

print(f'Total Mean IoU: {np.mean(ious_move_to_data)}')

In [ ]:
# on évalue les modifications apportées au modèle à l'aide des images de test
model.evaluate(image, lab)  

## Fine-tuning
Pour comparer la méthode move-to-data on implémente une méthode déjà éprouvée, le fine-tuning consiste à réentrainer la dernière couche du réseau avec de nouveaux batch d'image

In [ ]:
# on recharge le modèle pour ne pas écraser celui modifié par le move-to-data
model_fine = tf.keras.models.load_model('../models/best_models', custom_objects=None, compile=True, options=None)

ious_fine_tuning = []
i = 0
# Pour toutes les vidéos qui sont dans le jeu de test
for file in listdir("../GT_test"):
    if file.endswith(".txt"):
        i += 1
        print(f'Video {i} on {len(listdir("../GT_test"))}')
        
        m = re.search('(.+)_._bboxes\.txt', file)
        name = m.group(1)
        
        path_video = path.join('..', 'VIDEOS', name + '.mp4')
        path_boxes = path.join('..', 'GT_test', file)
        
        label = get_label(path_video)
        
        # on lance le tracking en activant le fine-tuning, on met à jour le réseau toutes les 10 images
        predictions, iou = tracking(model_fine, path_video, path_boxes, label, update='fine-tuning', epsilon=0.0001, n_update=10, save_fig=False)
        mean_iou = np.mean(list(iou.values()))
        ious_fine_tuning.append(mean_iou)
        
print(f'Mean IoU: {np.mean(ious_fine_tuning)}')

In [ ]:
# on évalue les modifications apportées au modèle à l'aide des images de test
model_fine.evaluate(image, lab)  

## Comparaisons
Dans cette partie, on compare les résultats que nous avons obtenu en tracking simple (sans mise à jour) avec les résultats que nous aurions obtenu en utilisant les modèles fournis. Cette comparaison ne sera faite que sur une seule vidéo du fait d'un temps de calcul relativement long.

In [ ]:
model = tf.keras.models.load_model('../models/mobilenet_based__16.h5', custom_objects=None, compile=True, options=None)
label = get_label('../VIDEOS/RicePlace6Subject4.mp4')
_, iou = tracking(model, '../VIDEOS/RicePlace6Subject4.mp4', '../GT_test/RicePlace6Subject4_2_bboxes.txt', label, save_fig=True, name='mobilenet')
mean_iou = np.mean(list(iou.values()))

print(f'Mean IoU: {mean_iou}')

In [ ]:
model = tf.keras.models.load_model('../models/mobilenetv2_based__6.h5', custom_objects=None, compile=True, options=None)
label = get_label('../VIDEOS/RicePlace6Subject4.mp4')
_, iou = tracking(model, '../VIDEOS/RicePlace6Subject4.mp4', '../GT_test/RicePlace6Subject4_2_bboxes.txt', label, save_fig=True, name='mobilenetV2')
mean_iou = np.mean(list(iou.values()))

print(f'Mean IoU: {mean_iou}')

### OpenCV Tracker
La bibliothèque OpenCV propose nativement des trackers que nous avons également comparé au notre. Nous en avons essayé trois, qui présentent des performances différentes.

In [ ]:
tracker_open_cvmodel('../VIDEOS/RicePlace6Subject4.mp4', '../GT_test/RicePlace6Subject4_2_bboxes.txt', 'kcf')

In [ ]:
tracker_open_cvmodel('../VIDEOS/RicePlace6Subject4.mp4', '../GT_test/RicePlace6Subject4_2_bboxes.txt', 'csrt')

In [ ]:
tracker_open_cvmodel('../VIDEOS/RicePlace6Subject4.mp4', '../GT_test/RicePlace6Subject4_2_bboxes.txt', 'mil')